# 모의고사 1회
## 작업 1유형

- 데이터 출처: https://archive.ics.uci.edu/ml/datasets/Bank+Marketing (후처리 작업)
- 데이터 설명: 은행의 전화 마케팅에 대해 고객의 반응 여부
- dataurl: https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/train.csv

In [20]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/train.csv')
df.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,y
0,13829,29,technician,single,tertiary,no,18254,no,no,cellular,11,may,2,-1,0,unknown,no
1,22677,26,services,single,secondary,no,512,yes,yes,unknown,5,jun,3,-1,0,unknown,no
2,10541,30,management,single,secondary,no,135,no,no,cellular,14,aug,2,-1,0,unknown,no
3,13689,41,technician,married,unknown,no,30,yes,no,cellular,10,jul,1,-1,0,unknown,no
4,11304,27,admin.,single,secondary,no,321,no,yes,unknown,2,sep,1,-1,0,unknown,no


Q. 마케팅 응답 고객들의 나이를 10살 단위로 변환 했을 때, 가장 많은 인원을 가진 나이대는? (0 ~ 9 : 0 , 10 ~ 19 : 10)

In [21]:
(df['age'] // 10 * 10).value_counts().index[0]

30

Q. 마케팅 응답 고객들의 나이를 10살 단위로 변환 했을 때, 가장 많은 나이대 구간의 인원은 몇명인가?

In [22]:
(df['age'] // 10 * 10).value_counts().max()

5056

Q. 나이가 25살 이상 29살 미만인 응답 고객들중 housing컬럼의 값이 yes인 고객의 수는?

In [23]:
df[(df['age'] >= 25) & (df['age'] < 29)]['housing'].value_counts().values[0]

504

Q. numeric한 값을 가지지 않은 컬럼들중 unique한 값을 가장 많이 가지는 컬럼은?

In [24]:
answer = df.select_dtypes(exclude='int').nunique().sort_values(ascending=False).index[0]
print(answer)

job


Q. balance 컬럼값들의 평균값 이상을 가지는 데이터를 ID값을 기준으로 내림차순 정렬했을때 상위 100개 데이터의 balance값의 평균은?

In [25]:
df[df['balance'] >= df['balance'].mean()].sort_values('ID', ascending=False)[:100]['balance'].mean()

3473.73

Q. 가장 많은 광고를 집행했던 날짜는 언제인가? (데이터 그대로 일(숫자),달(영문)으로 표기)

In [26]:
df.value_counts(['day', 'month']).index[0]

(15, 'may')

Q. 데이터의 job이 unknown 상태인 고객들의 age 컬럼 값의 정규성을 검정하고자 한다. 샤피로 검정의 p-value값을 구하여라

In [27]:
target = df[df['job'] == 'unknown']['age']
from scipy.stats import shapiro
stats, pvalue = shapiro(target)
pvalue

0.19612720608711243

Q. age와 balance의 상관계수를 구하여라

In [28]:
df[['age', 'balance']].corr().iloc[0, 1]

0.10198734763981462

Q. y 변수와 education 변수는 독립인지 카이제곱검정을 통해 확인하려한다. p-value값을 출력하라

In [29]:
from scipy.stats import chi2_contingency
target = pd.crosstab(df['y'], df['education'])
stats, pvalue, dof, expected = chi2_contingency(target)
pvalue

7.901201277473551e-29

Q. 각 job에 따라 divorced/married 인원의 비율을 확인 했을 때 그 값이 가장 높은 값은?

In [30]:
target = df[df['marital'] != 'single'].groupby('job')['marital'].value_counts().unstack().reset_index().fillna(0)
target['ratio'] = target.divorced / target.married
target['ratio'].max()

0.2831050228310502

## 작업 2유형
- 데이터 출처: https://archive.ics.uci.edu/ml/datasets/Bank+Marketing (후처리 작업)
- 데이터 설명: 은행의 전화 마케팅에 대해 고객의 반응 여부
- train: https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/train.csv
- test: https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/test.csv
- submission: https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/submission.csv

In [31]:
import pandas as pd
train= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/train.csv')
test= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/test.csv')
submission= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/submission.csv')

x_train = train[train.columns[1:-1]]
x_test = test[test.columns[1:]]
y_train = train['y']

x_train = pd.get_dummies(x_train)
x_test = pd.get_dummies(x_test)

from sklearn.preprocessing import StandardScaler
scaled_cols = ['age', 'balance', 'campaign', 'pdays', 'previous', 'day']
scaler = StandardScaler()
x_train[scaled_cols] = scaler.fit_transform(x_train[scaled_cols])
x_test[scaled_cols] = scaler.transform(x_test[scaled_cols])

from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(x_train, y_train, stratify=y_train, test_size=0.2, random_state=10)

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=10, max_depth=15)
model.fit(X_tr, y_tr)

tr_predict = model.predict_proba(X_tr)[:, 1]
val_predict = model.predict_proba(X_val)[:, 1]

from sklearn.metrics import roc_auc_score
print('train auc:', roc_auc_score(y_tr, tr_predict))
print('val auc:', roc_auc_score(y_val, val_predict))

test_predict = model.predict_proba(x_test)[:, 1]
pd.DataFrame({'ID': test.ID, 'predict': test_predict})

train auc: 0.9594122209251771
val auc: 0.802483550267536


,ID,predict
0,53608,0.792265
1,51055,0.740612
2,52573,0.080598
3,50458,0.110771
4,52272,0.245811
...,...,...
4286,50332,0.182475
4287,51293,0.797865
4288,54023,0.232057
4289,53775,0.582532


# 모의고사 2회
## 작업 1유형

- 데이터 출처: https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset (후처리 작업)
- 데이터 설명: 뇌졸증 발생여부 예측
- dataurl: https://raw.githubusercontent.com/Datamanim/datarepo/main/stroke/train.csv

In [32]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/stroke_/train.csv')
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1192,Female,31,0,0,No,Govt_job,Rural,70.66,27.2,never smoked,0
1,77,Female,13,0,0,No,children,Rural,85.81,18.6,Unknown,0
2,59200,Male,18,0,0,No,Private,Urban,60.56,33.0,never smoked,0
3,24905,Female,65,0,0,Yes,Private,Urban,205.77,46.0,formerly smoked,1
4,24257,Male,4,0,0,No,children,Rural,90.42,16.2,Unknown,0


Q. 성별이 Male인 환자들의 age의 평균값은 ?

In [33]:
df['age'] = df['age'].str.replace('*', '').astype('int')
answer = df[df['gender'] == 'Male']['age'].mean()
print(answer)

44.68623481781376


<ipython-input-33-7277120a4904>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['age'] = df['age'].str.replace('*', '').astype('int')


Q. bmi컬럼의 결측치를 bmi컬럼의 결측치를 제외한 나머지 값들의 중앙값으로 채웠을 경우 bmi 컬럼의 평균을 소숫점 이하 3자리 까지 구하여라

In [34]:
target = df['bmi'].fillna(df['bmi'].median())
round(target.mean(), 3)

29.166

Q. bmi컬럼의 각 결측치들을 직전의 행의 bmi값으로 채웠을 경우 bmi 컬럼의 평균을 소숫점 이하 3자리 까지 구하여라

In [35]:
target = df['bmi'].fillna(method='ffill')
round(target.mean(), 3)

29.188

Q. bmi컬럼의 각 결측치들을 결측치를 가진 환자 나이대(10단위)의 평균 bmi 값으로 대체한 후 대체된 bmi 컬럼의 평균을 소숫점 이하 3자리 까지 구하여라

In [36]:
target = df[['age', 'bmi']].copy()
target['age'] = target['age'] // 10 * 10
mean = target.groupby('age')['bmi'].mean()
dic = {x: y for x, y in mean.items()}

target.loc[target['bmi'].isnull(), 'bmi'] = target[target['bmi'].isnull()]['age'].map(lambda x: dic[x])
round(target['bmi'].mean(), 3)

29.263

Q. avg_glucose_level 컬럼의 값이 200이상인 데이터를 모두 199로 변경하고 stroke값이 1인 데이터의 avg_glucose_level값의 평균을 소수점이하 3자리 까지 구하여라

In [37]:
df.loc[df['avg_glucose_level'] >= 200, 'avg_glucose_level'] = 199
round(df[df['stroke'] == 1]['avg_glucose_level'].mean(), 3)

125.188

## 작업 1유형 - 다른 데이터

- 데이터 출처: https://www.kaggle.com/abcsds/pokemon (참고, 데이터 수정)
- 데이터 설명: 포켓몬 정보
- data url: https://raw.githubusercontent.com/Datamanim/datarepo/main/pok/Pokemon.csv

In [38]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/pok/Pokemon.csv')
df.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


Q. Attack컬럼의 값을 기준으로 내림차순정렬 했을때 상위 400위까지 포켓몬들과 401~800위까지의 포켓몬들에서 전설포켓몬(Legendary컬럼)의 숫자 차이는?

In [48]:
top400 = df.sort_values('Attack', ascending=False).head(400)
top800 = df.sort_values('Attack', ascending=False).tail(400)
top400.value_counts('Legendary').values[1] - top800.value_counts('Legendary').values[1]

57

Q. Type 1 컬럼의 종류에 따른 Total 컬럼의 평균값을 내림차순 정렬했을때 상위 3번째 Type 1은 무엇인가?

In [51]:
df.groupby('Type 1')['Total'].mean().sort_values(ascending=False).index[2]

'Flying'

Q. 결측치가 존재하는 행을 모두 지운 후 처음부터 순서대로 60% 데이터를 추출하여 Defense컬럼의 1분위수를 구하여라

In [59]:
target = df.dropna(axis=0)
target[:int(len(target)*0.6)]['Defense'].quantile(0.25)

50.0

Q. Type 1 컬럼의 속성이 Fire인 포켓몬들의 Attack의 평균이상인 Water속성의 포켓몬 수를 구하여라

In [65]:
mean = df[df['Type 1'] == 'Fire']['Attack'].mean()
df[(df['Type 1'] == 'Water') & (df['Attack'] >= mean)].shape[0]

37

Q. 각 세대 중(Generation 컬럼)의 Speed와 Defense 컬럼의 차이(절댓값)이 가장 큰 세대는?

In [85]:
target = df.groupby('Generation')[['Speed', 'Defense']].mean()
abs(target['Speed'] - target['Defense']).sort_values(ascending=False).index[0]

2

## 작업 2유형
- 데이터 출처: https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset (후처리 작업)
- 데이터 설명: 뇌졸증 발생여부 예측
- train: https://raw.githubusercontent.com/Datamanim/datarepo/main/stroke_/train.csv
- test: https://raw.githubusercontent.com/Datamanim/datarepo/main/stroke_/test.csv

In [175]:
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
train= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/stroke_/train.csv')
test= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/stroke_/test.csv')

idx = train[train['gender'] == 'Other'].index
train = train.drop(index=idx)

x_train = train[train.columns[1:-1]]
x_test = test[test.columns[1:]]
y_train = train.stroke

x_train['age'] = x_train['age'].str.replace('*', '').astype('int32')
x_test['age'] = x_test['age'].astype('int32')

fill = x_train['bmi'].mean()
x_train['bmi'] = x_train['bmi'].fillna(fill)
x_test['bmi'] = x_test['bmi'].fillna(fill)

from sklearn.preprocessing import LabelEncoder
cols = ['gender', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']
encoder = LabelEncoder()
for col in cols:
    x_train[col] = encoder.fit_transform(x_train[col])
    x_test[col] = encoder.fit_transform(x_test[col])

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(x_train, y_train, test_size=0.25, stratify=y_train, random_state=10)

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=10, max_depth=14, n_estimators=200)
model.fit(X_tr, y_tr)

tr_predict = model.predict_proba(X_tr)[:, 1]
val_predict = model.predict_proba(X_val)[:, 1]
te_predict = model.predict_proba(x_test)[:, 1]

from sklearn.metrics import roc_auc_score
print('tr auc:', roc_auc_score(y_tr, tr_predict))
print('val auc:', roc_auc_score(y_val, val_predict))

pd.DataFrame({'id': test.id, 'predict': te_predict})

tr auc: 1.0
val auc: 0.8530588235294119


,id,predict
0,47472,0.168466
1,36841,0.275000
2,3135,0.355000
3,65218,0.000000
4,1847,0.015000
...,...,...
927,40311,0.181176
928,44281,0.005000
929,11238,0.006186
930,17308,0.337500


# 모의고사 3회
## 작업 1유형

- 데이터 출처: link (후처리 작업)
- 데이터 설명: 2010-2019 스포티파이 TOP100 노래
- dataurl: https://raw.githubusercontent.com/Datamanim/datarepo/main/spotify/spotify.csv

In [176]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/spotify/spotify.csv')
df.head()

,title,artist,top genre,year released,added,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop,top year,artist type
0,STARSTRUKK (feat. Katy Perry),3OH!3,dance pop,2009.0,2022‑02‑17,140.0,81.0,61.0,-6.0,23.0,23.0,203.0,0.0,6.0,70.0,2010.0,Duo
1,My First Kiss (feat. Ke$ha),3OH!3,dance pop,2010.0,2022‑02‑17,138.0,89.0,68.0,-4.0,36.0,83.0,192.0,1.0,8.0,68.0,2010.0,Duo
2,I Need A Dollar,Aloe Blacc,pop soul,2010.0,2022‑02‑17,95.0,48.0,84.0,-7.0,9.0,96.0,243.0,20.0,3.0,72.0,2010.0,Solo
3,Airplanes (feat. Hayley Williams of Paramore),B.o.B,atl hip hop,2010.0,2022‑02‑17,93.0,87.0,66.0,-4.0,4.0,38.0,180.0,11.0,12.0,80.0,2010.0,Solo
4,Nothin' on You (feat. Bruno Mars),B.o.B,atl hip hop,2010.0,2022‑02‑17,104.0,85.0,69.0,-6.0,9.0,74.0,268.0,39.0,5.0,79.0,2010.0,Solo


Q. 데이터는 현재 년도별 100곡이 인기순으로 정렬되어 있다. 각 년도별 1~100위의 랭킹을 나타내는 rank컬럼을 만들고 매년도 1위의 bpm컬럼의 평균값을 구하여라

In [184]:
df = df.dropna()
df['rank'] = list(range(1, 101))*10
df[df['rank'] == 1].bpm.mean()

125.6

Q. 2015년도에 가장많은 top100곡을 올린 artist는 누구인가?

In [187]:
df[df['top year'] == 2015.0]['artist'].value_counts().index[0]

'Taylor Swift'

Q. 년도별 rank값이 1~10위 까지의 곡들 중 두번째로 많은 top genre는 무엇인가?

In [194]:
df[df['rank'] <= 10]['top genre'].value_counts().index[1]

'atl hip hop'

Q. 피처링의 경우 title에 표시된다. 피처링을 가장 많이 해준 가수는 누구인가?

In [207]:
df['title'].apply(lambda x: x.split('feat.')[-1].replace(')', '')).value_counts().index[0]

' Bruno Mars'

Q. top year 년도를 기준으로 발매일(year released)과 top100에 진입한 일자 (top year)가 다른 곡의 숫자를 count 했을때 가장 많은 년도는?

In [213]:
df[df['top year'] != df['year released']]['top year'].value_counts().index[0]

2016.0

Q. artist 컬럼의 값에 대소문자 상관없이 q 단어가 들어가는 아티스트는 몇명인가?

In [216]:
df[df['artist'].str.lower().str.contains('q')]['artist'].nunique()

6

Q. 년도 상관없이 전체데이터에서 1 ~ 50위와 51 ~ 100위간의 dur 컬럼의 평균값의 차이는?

In [217]:
df[df['rank'] <= 50]['dur'].mean() - df[df['rank'] > 50]['dur'].mean()

0.896000000000015

Q. title을 띄어쓰기 단어로 구분 했을때 가장 많이 나온 단어는 무엇인가? (대소문자 구분 x)

In [235]:
target = df['title'].str.replace('(', '').str.replace(')', '').str.lower().str.split(' ')
aList = []
for lis in target:
    for li in lis:
        aList.append(li)
pd.Series(aList).value_counts().index[0]

'feat.'

Q. 년도별 nrgy값의 평균값을 구할때 최대 평균값과 최소 평균값의 차이를 구하여라

In [237]:
mean = df.groupby('top year')['nrgy'].mean().sort_values().values
mean.max() - mean.min()

13.860000000000007

Q. artist중 artist type 타입을 여러개 가지고 있는 artist는 누구인가

In [250]:
target = df[['artist', 'artist type']].value_counts().reset_index()['artist'].value_counts()
target[target > 1].index[0]

'Rudimental'

## 작업 2유형
- 데이터 설명: 센서데이터로 동작 유형 분류 (종속변수 pose : 0 ,1 구분)
- x_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/muscle/x_train.csv
- y_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/muscle/y_train.csv
- x_test: https://raw.githubusercontent.com/Datamanim/datarepo/main/muscle/x_test.csv
출처(참고, 데이터 수정)

In [265]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/muscle/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/muscle/y_train.csv")
test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/muscle/x_test.csv")

x_train = x_train.drop(columns='ID')
y_train = y_train.drop(columns='ID')
x_test = test.drop(columns='ID')

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(x_train, y_train, stratify=y_train, test_size=0.25, random_state=10)

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=10)
model.fit(X_tr, y_tr)

tr_predict = model.predict_proba(X_tr)[:, 1]
te_predict = model.predict_proba(X_te)[:, 1]
predict = model.predict_proba(x_test)[:, 1]

from sklearn.metrics import roc_auc_score
print('train auc:', roc_auc_score(y_tr, tr_predict))
print('test auc:', roc_auc_score(y_te, te_predict))

pd.DataFrame({'ID': test.ID, 'pose': predict})

train auc: 1.0
test auc: 0.9999807773065753


,ID,pose
0,1,1.00
1,3,1.00
2,8,1.00
3,10,1.00
4,17,0.00
...,...,...
1158,5786,0.03
1159,5796,0.00
1160,5802,1.00
1161,5811,1.00
